# DBS - Praktikum 1 - 4. November 2024

## Aufgabe 1: Datenbank Account

Verbindung zum Server:

```bash
psql -U dbs<User> -h 194.94.121.224 -p 15434 dbs<DB Name>
```

Anpassen des Passworts in der Datenbank:

```sql
ALTER USER dbs<User> PASSWORD '<password>';
```

## Aufgabe 2: Hello World

1. Erzeugen einer Textdatei `hello.sql` mit dem Inhalt `SELECT 'Hello World';`:

```bash
\! echo "SELECT 'Hello World';" > hello.sql
```

2. Ausführen des SQL-Skripts:

```bash
\i hello.sql
```

## Aufgabe 3: Arbeiten mit der psql-Shell

1. Anlegen einer beliebigen Tabelle mit nur einer Spalte

```sql
CREATE TABLE beispiel (id INT PRIMARY KEY);
```

2. Anzeigen der Tabellenstruktur

```sql
\d beispiel
```

3.  SQL Script zum Einfügen von Daten in die Tabelle in dem Script `insert.sql` speichern:

```bash
echo "INSERT INTO beispiel VALUES (1);" > insert.sql
```

```sql
\i insert.sql
```

4. Entfernen der Tabelle

```sql
DROP TABLE beispiel;
```

## Aufgabe 4: Tabellen anlegen und Daten-Import

1. Anlange der Tabellen `Import`,  `Kunde` und `Bestellung`:

```sql
CREATE TABLE Import (
    BestellID VARCHAR(20),
    KundenID VARCHAR(20),
    Bestelldatum VARCHAR(10),
    Betrag VARCHAR(20),
    Status VARCHAR(50),
    Titel VARCHAR(20),
    Vorname VARCHAR(50),
    Name VARCHAR(50),
    PLZ VARCHAR(10),
    Ort VARCHAR(100)
);

CREATE TABLE Kunde (
    KundenID INT PRIMARY KEY,
    Titel VARCHAR(20),
    Vorname VARCHAR(50) NOT NULL,
    Name VARCHAR(50) NOT NULL,
    PLZ CHAR(5),
    Ort VARCHAR(100) NOT NULL
);

CREATE TABLE Bestellung (
    BestellID VARCHAR(20) PRIMARY KEY,
    KundenID INT NOT NULL,
    Bestelldatum DATE NOT NULL,
    Betrag DECIMAL(10, 2) NOT NULL,
    Status VARCHAR(20) NOT NULL,
    FOREIGN KEY (KundenID) REFERENCES Kunde(KundenID) ON DELETE CASCADE
);

```

2. Importieren der Daten aus der Datei `p1-data.csv`:

```sql
\copy import FROM 'p1-data.csv' WITH (FORMAT csv, DELIMITER ';', HEADER true, ENCODING 'UTF8');
```

3. Prüfen ob die Daten korrekt importiert wurden:

```sql
SELECT * FROM Import;
```

Falls es Encoding Probleme geben sollte: `\encoding UTF8`

4. Einfügen der Daten in Tabelle `Kunde`:

```sql
INSERT INTO Kunde (KundenID, Titel, Vorname, Name, PLZ, Ort)
SELECT DISTINCT KundenID::INT, Titel, Vorname, Name, PLZ, Ort
FROM Import;
```

5. Einfügen der Daten in Tabelle `Bestellung`:

```sql
INSERT INTO Bestellung (BestellID, KundenID, Bestelldatum, Betrag, Status)
SELECT BestellID, KundenID::INT, TO_DATE(Bestelldatum, 'YYYY-MM-DD'), CAST(Betrag AS DECIMAL(10, 2)), Status
FROM Import;
```

**Warum benötigt man `SELECT DISTINCT`?**
SELECT DISTINCT verhindert, dass ein Kunde der doppelt bestellt hat, auch doppelt in der Tabelle Kunde eingefügt wird. Somit taucht jeder Kunde nur einmal in der Tabelle auf.


## Aufgabe 5: Änderungen und Abfragen

1. Neue Bestellung einfügen:

Daten: Kunde: Julie Schuster, Betrag: 49,95€, Bestelldatum: 14.10.2024, Status: In Bearbeitung

```sql
INSERT INTO Kunde (KundenID, Titel, Vorname, Name, PLZ, Ort)
VALUES (123,'', 'Julie', 'Schuster', '12345', 'Berlin');

INSERT INTO Bestellung (BestellID, KundenID, Bestelldatum, Betrag, Status)
VALUES ('B222', 123, '2024-10-14', 49.95, 'In Bearbeitung');
```

2. Alle Bestellung mit dem Status "Storniert" entfernen:

```sql
DELETE FROM Bestellung
WHERE Status = 'Storniert';
```

3. Alle Bestellungen vor dem 1. Juni 2024 die den Status "In Bearbeitung" haben, sollen den Status auf "Abgeschlossen" geändert werden:

```sql
UPDATE Bestellung
SET Status = 'Abgeschlossen'
WHERE Bestelldatum < '2024-06-01' AND Status = 'In Bearbeitung';
```

4. Welche Bestellungen haben einen Wert größer als 400€? Die Liste soll nach dem Bestellwert absteigen sortiert sein:

```sql
SELECT * FROM Bestellung
WHERE Betrag > 400
ORDER BY Betrag DESC;
```

5. Aus welchen Orten kommen die Kunden, die im Juni 2024 Bestellungen aufgegeben haben? 

```sql
SELECT DISTINCT Ort FROM Kunde
JOIN Bestellung ON Kunde.KundenID = Bestellung.KundenID
WHERE Bestelldatum >= '2024-06-01' AND Bestelldatum < '2024-07-01';
```

5.1 Einfachere Variante: Aus welchen Orten kommen Kunden, die den Titel 'Dr.' oder 'Prof. Dr.' haben?

```sql
SELECT DISTINCT Ort FROM Kunde
WHERE Titel IN ('Dr.', 'Prof. Dr.');
```

6. Geben Sie für jede Stadt aus, wie viele Bestellungen von den Kunden aus dieser Stadt aufgegeben wurden.

```sql
SELECT Ort, COUNT(BestellID) AS Anzahl_Bestellungen
FROM Kunde
JOIN Bestellung ON Kunde.KundenID = Bestellung.KundenID
GROUP BY Ort;
```

6.1 Einfachere Variante: Geben Sie für jede Stadt aus, wie viele Kunden aus der Stadt kommen.

```sql
SELECT Ort, COUNT(KundenID) AS Anzahl_Kunden
FROM Kunde
GROUP BY Ort;
```
